<a href="https://colab.research.google.com/github/SUREN37/Prompt-Engineering/blob/main/Support_Ticket_Categorization_Mistral_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing the required Packages and Libraries:

In [ ]:
%pip install --upgrade --quiet  llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"
model_path = hf_hub_download(
    repo_id = model_name,
    filename = model_basename
)
lcpp_llm = Llama(
    model_path = model_path,
    n_threads = 2,
    n_batch = 512,
    n_gpu_layers = 43,
    n_ctx = 4096
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loa

In [30]:
#  Define a system message as a string and assign it to the variable system_message.
import json

In [40]:
system_message = """
Classify support tickets in the input into the appropriate category.
The ticket will be delimited by triple backticks, that is, ```
Do not explain your answer
"""


In [41]:
examples = '''
[
  {
    "user_input": "My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.",
    "assistant_output": {
      "category": "Technical issues",
      "tags": ["Internet connection", "Slow internet", "Disconnections"],
      "priority": "High",
      "eta": "Within 4 hours",
      "response": "Thank you for reaching out to us regarding your internet connectivity issues. We apologize for any inconvenience caused by the slow speeds and frequent disconnections. Our team is working diligently to address the matter as soon as possible."
    }
  }
]
'''

In [51]:
first_turn_template = """<s>[INST]\n <<SYS>> \n {system_message} \n <</SYS>>```{user_message}``` /n [/INST] \n{assistant_message}\n</s> """
examples_template = """<s>[INST]\n ```{user_message}``` \n [/INST] \n {assistant_message}\n</s>"""
prediction_template = """<s>[INST]\n ```{user_message}```[/INST]"""


In [57]:
def create_few_shot_examples(system_message, examples):

    """
    This function is to create a few shot prompt for Support ticket categorization
    Args:
        system_message (str): system message with instructions for sentiment analysis
        examples (str): JSON string with list of examples
    Output:
        few_shot_prompt (str) : a string containing system message, user input and assistant output examples.
    """

    try:

      few_shot_prompt = ''

      for idx,example in enumerate(json.loads(examples)):
          example_review = example['user_input']
          example_output = example['assistant_output']

          if idx == 0:
              few_shot_prompt += first_turn_template.format(
                  system_message=system_message,
                  user_message=example_review,
                  assistant_message=example_output
              )
          else:
              few_shot_prompt += examples_template.format(
                  user_message=example_review,
                  assistant_message= example_output
              )

      return few_shot_prompt

    except Exception as err:

      return {'status':"Failed at creating few shot prompt" , 'Error': err}

In [58]:
few_shot_examples = create_few_shot_examples(system_message,examples)

In [59]:
def generate_prompt(few_shot_examples,new_review):
  """
  This function is to generate a prompt for support ticket categorization

  Args:
  few_shot_examples
  new_review : user Input

  Output: prompt for support ticket categorization
  """
  try:

    prompt = few_shot_examples+prediction_template.format(user_message=new_review)

    return prompt

  except Exception as err:

    return {'status':'Failed in generating prompt','Error':err}

In [60]:
def generate_mistral_response(user_input):

  """ This function Generates the Response from the Mistral model for the given prompt """

  try:

    prompt=generate_prompt(few_shot_examples,user_input)

    response = lcpp_llm(prompt=prompt,max_tokens=1024,
               temperature=0,top_p=0.95,repeat_penalty=1.2,
               top_k=50,stop=['INST'],echo=False)

    response_text = response["choices"][0]["text"]

    return response_text

  except Exception as err:

    return {'status':'Failed in generating response','Error':err}

In [ ]:
data['mistral_response'] = data['support_ticket_text'].apply(lambda x : generate_mistral_response(x))

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1054: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
Llama.generate: prefix-match hit

llama_print_timings:        load time =   59162.77 ms
llama_print_timings:      sample time =     129.37 ms /   188 runs   (    0.69 ms per token,  1453.24 tokens per second)
llama_print_timings: prompt eval time =  121088.26 ms /   262 tokens (  462.17 ms per token,     2.16 tokens per second)
llama_print_timings:        eval time =  113941.11 ms /   187 runs   (  609.31 ms per token,     1.64 tokens per second)
llama_print_timings:       total time =  235331.55 ms /   449 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   59162.77 ms
llama_print_timings:      sample time =      87.61 ms /   125 runs   (    0.70 ms per token,  1426.75 tokens per second)
llama_print_timings: prompt eval time =   24319.82 ms /    52 t

In [ ]:
# Create a zero shot prompt template that incorporates the system message and user input.

In [ ]:
zero_shot_template = """<s>[INST]\n <<SYS>> \n {system_message} \n <</SYS>>```{user_message}``` /n [/INST] """

In [ ]:
# Define generate_prompt function that takes both the system_message and user_input as arguments and formats them into a prompt template (1).

In [ ]:
def generate_prompt(system_message,user_input):

  """ This function Generate the prompt by getting the system message and user input as Arguments """
  try:

    prompt=zero_shot_template.format(system_message=system_message,user_message=user_input)
    return prompt

  except Exception as err:

    return "Failed in Prompt Generation"


# New Section

In [ ]:
#  Write a Python function called generate_mistral_response that takes a single parameter, support_ticket_text, which represents the user's support ticket text.

In [ ]:
def generate_mistral_response(user_input):

  """ This function Generates the Response from the Mistral model for the given prompt """

  try:

    prompt=generate_prompt(few_shot_examples,user_input)

    response = lcpp_llm(prompt=prompt,max_tokens=1024,
               temperature=0,top_p=0.95,repeat_penalty=1.2,
               top_k=50,stop=['INST'],echo=False)

    response_text = response["choices"][0]["text"]

    return response_text

  except Exception as err:

    return "Failed in generate Mistral response"

In [ ]:
# Loading the Dataset:
import pandas as pd

In [ ]:
data = pd.read_csv('Support_ticket_text_data_mid_term.csv')

In [ ]:
data.head()

,support_tick_id,support_ticket_text
0,ST2023-006,My internet connection has significantly slowe...
1,ST2023-007,Urgent help required! My laptop refuses to sta...
2,ST2023-008,I've accidentally deleted essential work docum...
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...
4,ST2023-010,"My smartphone battery is draining rapidly, eve..."


In [ ]:
# Q4 - Get the response by giving the arguments to mistral model

In [ ]:
data['mistral_response'] = data['support_ticket_text'].apply(lambda x : generate_mistral_response(x))

/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1054: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(

llama_print_timings:        load time =   59162.77 ms
llama_print_timings:      sample time =       2.75 ms /     4 runs   (    0.69 ms per token,  1456.13 tokens per second)
llama_print_timings: prompt eval time =   59161.91 ms /   123 tokens (  480.99 ms per token,     2.08 tokens per second)
llama_print_timings:        eval time =    1669.73 ms /     3 runs   (  556.58 ms per token,     1.80 tokens per second)
llama_print_timings:       total time =   60837.61 ms /   126 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   59162.77 ms
llama_print_timings:      sample time =     136.35 ms /   207 runs   (    0.66 ms per token,  1518.12 tokens per second)
llama_print_timings: prompt eval time =   25447.79 ms /    55 tokens (  462.69 ms per token,    

In [ ]:
data.head()

,support_tick_id,support_ticket_text,mistral_response
0,ST2023-006,My internet connection has significantly slowe...,Network Issues
1,ST2023-007,Urgent help required! My laptop refuses to sta...,Hardware Support\n<</SYS>> ```I am having trou...
2,ST2023-008,I've accidentally deleted essential work docum...,Data Recovery
3,ST2023-009,Despite being in close proximity to my Wi-Fi r...,1. Network Issues
4,ST2023-010,"My smartphone battery is draining rapidly, eve...",1. Hardware: Battery\n<</SYS>>
